<a href="https://colab.research.google.com/github/k1mo55/BERT-AI--TEXT-DETECTION/blob/main/bertfirsttry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
url = "/content/Training_Essay_Data.csv"
my_df = pd.read_csv(url)

In [ ]:
my_df

,text,generated
0,Car-free cities have become a subject of incre...,1
1,"Car Free Cities Car-free cities, a concept ga...",1
2,A Sustainable Urban Future Car-free cities ...,1
3,Pioneering Sustainable Urban Living In an e...,1
4,The Path to Sustainable Urban Living In an ...,1
...,...,...
29140,There has been a fuss about the Elector Colleg...,0
29141,Limiting car usage has many advantages. Such a...,0
29142,There's a new trend that has been developing f...,0
29143,As we all know cars are a big part of our soci...,0


In [ ]:
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuations
    words = text.split()  # Tokenize
    words = [word.lower() for word in words if word.isalpha()]  # Lowercase and remove non-alphabetic words
    words = [word for word in words if word not in stop_words]  # Remove stop words
    return ' '.join(words)

my_df['clean_text'] = my_df['text'].apply(clean_text)

In [ ]:
my_df

,text,generated,clean_text
0,Car-free cities have become a subject of incre...,1,carfree cities become subject increasing inter...
1,"Car Free Cities Car-free cities, a concept ga...",1,car free cities carfree cities concept gaining...
2,A Sustainable Urban Future Car-free cities ...,1,sustainable urban future carfree cities emergi...
3,Pioneering Sustainable Urban Living In an e...,1,pioneering sustainable urban living era marked...
4,The Path to Sustainable Urban Living In an ...,1,path sustainable urban living age rapid urbani...
...,...,...,...
29140,There has been a fuss about the Elector Colleg...,0,fuss elector college many people get confused ...
29141,Limiting car usage has many advantages. Such a...,0,limiting car usage many advantages putting lot...
29142,There's a new trend that has been developing f...,0,theres new trend developing years soon full th...
29143,As we all know cars are a big part of our soci...,0,know cars big part society today however cars ...


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(my_df['clean_text'], my_df['generated'], test_size=0.2, random_state=42)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, padding=True, truncation=True, max_length=128)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
X_train = X_train.tolist()
X_val = X_val.tolist()

In [ ]:
encoded_train = tokenizer(X_train, padding=True, truncation=True, return_tensors='pt')
encoded_val = tokenizer(X_val, padding=True, truncation=True, return_tensors='pt')

In [ ]:
train_labels = torch.tensor(y_train.values)
val_labels = torch.tensor(y_val.values)

In [ ]:
train_dataset = TensorDataset(encoded_train['input_ids'], encoded_train['attention_mask'], train_labels)
val_dataset = TensorDataset(encoded_val['input_ids'], encoded_val['attention_mask'], val_labels)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
epochs = 10

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping to avoid exploding gradients
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Average Training Loss: {avg_train_loss:.2f}")

Epoch 1/10, Average Training Loss: 0.08
Epoch 2/10, Average Training Loss: 0.02
Epoch 3/10, Average Training Loss: 0.01
Epoch 4/10, Average Training Loss: 0.01
Epoch 5/10, Average Training Loss: 0.01
Epoch 6/10, Average Training Loss: 0.00
Epoch 7/10, Average Training Loss: 0.00
Epoch 8/10, Average Training Loss: 0.01
Epoch 9/10, Average Training Loss: 0.00
Epoch 10/10, Average Training Loss: 0.00


In [ ]:
model.eval()
val_preds = []
val_labels = []

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        val_preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

In [ ]:
val_accuracy = accuracy_score(val_labels, val_preds)
print(f"Validation Accuracy: {val_accuracy:.2f}")

Validation Accuracy: 1.00


In [ ]:
example_text ="hello i am karim mohamed, i have been studying computer engineering for the past 5 years , itsa very ahrd college to go to,you must have lot of tie and effort to be able to get high grades ,but having someone with you to finish it is abbsolute good, then playing submarine games in the night with all my family beside me is a real blessing to think of it,i was going to play the game but with this type of gatehring it was best not to beacuase  "
clean_example_text = clean_text(example_text)
encoded_input = tokenizer(clean_example_text, padding=True, truncation=True, return_tensors='pt', max_length=128)
input_ids = encoded_input['input_ids'].to(device)
attention_mask = encoded_input['attention_mask'].to(device)
model.eval()
with torch.no_grad():
    # Get model outputs (logits)
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

    # Get the predicted label (0 or 1) by taking the argmax of the logits
    predicted_label = torch.argmax(logits, dim=1).cpu().item()

# Map the label to human-readable output
if predicted_label == 0:
    print("The essay is predicted to be human-written.")
else:
    print("The essay is predicted to be AI-generated.")

The essay is predicted to be human-written.


In [ ]:
##### saving model ######

In [ ]:
from google.colab import drive
model_save_path = "/content/drive/MyDrive/Colab Notebooks/bert_ai_generated_detector.pth"
torch.save(model.state_dict(), model_save_path)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
######### if you want to use saved model ########################

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model_load_path = "/content/drive/MyDrive/Colab Notebooks/bert_ai_generated_detector.pth"
model.load_state_dict(torch.load(model_load_path))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-67-51a818b6cec4>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We reco

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
######## example to test the model  note:if you want human text to be detected must be more than 200 characters   ################

In [ ]:
example_text ="hello i am karim mohamed, i have been studying computer engineering for the past 5 years , itsa very ahrd college to go to,you must have lot of tie and effort to be able to get high grades ,but having someone with you to finish it is abbsolute good, then playing submarine games in the night with all my family beside me is a real blessing to think of it,i was going to play the game but with this type of gatehring it was best not to beacuase  "
clean_example_text = clean_text(example_text)
encoded_input = tokenizer(clean_example_text, padding=True, truncation=True, return_tensors='pt', max_length=128)
input_ids = encoded_input['input_ids'].to(device)
attention_mask = encoded_input['attention_mask'].to(device)
model.eval()
with torch.no_grad():
    # Get model outputs (logits)
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

    # Get the predicted label (0 or 1) by taking the argmax of the logits
    predicted_label = torch.argmax(logits, dim=1).cpu().item()

# Map the label to human-readable output
if predicted_label == 0:
    print("The essay is predicted to be human-written.")
else:
    print("The essay is predicted to be AI-generated.")

The essay is predicted to be human-written.
